In [3]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns

Definition

In [ ]:
entire_timeframe = 600 # second

# create figure and axis
fig, (axl, axr) = plt.subplots(ncols=2, figsize=(12,7), gridspec_kw=dict(width_ratios=[4,2], wspace = 0))
